In [ ]:
"""
🔍 CARA-WR: Quick Retrieval Demo
This notebook demonstrates writer retrieval on a trained model.
"""
import logging
from pathlib import Path
import sys

# Project root
ROOT = Path('..').resolve()
sys.path.append(str(ROOT))

# Setup logging
from src.utils.logging_utils import setup_logging
logger = setup_logging(name='cara_wr_demo', level=logging.INFO, use_colors=False)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%H:%M:%S',
    force=True
)

logger.info('=' * 60)
logger.info('🔍 CARA-WR: Writer Retrieval Demo')
logger.info('=' * 60)
logger.info('')

# ===== PATH CONFIGURATION =====
# Update these paths to match your experiment
DATASET = 'cvl'  # or 'iam'

if DATASET == 'cvl':
    TEST_CSV = ROOT / 'data' / 'cvl' / 'cvl_test.csv'
    CHECKPOINT_DIR = ROOT / 'experiments' / 'cvl' / 'checkpoint'
    LABELS_JSON = ROOT / 'data' / 'cvl' / 'cvl_labels.json'
elif DATASET == 'iam':
    TEST_CSV = ROOT / 'data' / 'iam' / 'iam_test.csv'
    CHECKPOINT_DIR = ROOT / 'experiments' / 'iam' / 'checkpoints'
    LABELS_JSON = ROOT / 'data' / 'iam' / 'iam_labels.json'

logger.info(f'📂 Dataset: {DATASET}')
logger.info(f'   Test CSV: {TEST_CSV}')
logger.info(f'   Checkpoints: {CHECKPOINT_DIR}')


In [ ]:
# === Run Retrieval Evaluation ===
import torch
from src.evaluation.eval_retrieval import main as eval_retrieval

# Find best checkpoint
ckpt_list = sorted(CHECKPOINT_DIR.glob('best_epoch*_loss*.pt'))

if ckpt_list:
    ckpt_path = ckpt_list[-1]
    logger.info(f'📂 Using checkpoint: {ckpt_path.name}')
    
    # Run evaluation with best settings
    eval_args = [
        '--csv', str(TEST_CSV),
        '--root-dir', str(ROOT),
        '--checkpoint', str(ckpt_path),
        '--agg-type', 'sum',      # Sum pooling
        '--mode', 'auto',         # Quantity-adaptive
        '--device', 'cuda',
        '--rerank', 'sgr',        # SGR reranking
    ]
    
    sys.argv = ['eval_retrieval.py'] + eval_args
    labels, paths, descs, metrics = eval_retrieval()
    
    logger.info('')
    logger.info('╔════════════════════════════════════╗')
    logger.info('║       RETRIEVAL RESULTS            ║')
    logger.info('╠════════════════════════════════════╣')
    logger.info(f'║  mAP:    {metrics["mAP"]*100:6.2f}%                  ║')
    logger.info(f'║  Top-1:  {metrics["Top1"]*100:6.2f}%                  ║')
    logger.info(f'║  Top-5:  {metrics["Top5"]*100:6.2f}%                  ║')
    logger.info(f'║  Top-10: {metrics["Top10"]*100:6.2f}%                  ║')
    logger.info('╚════════════════════════════════════╝')
else:
    logger.warning(f'⚠️ No checkpoint found in {CHECKPOINT_DIR}')


In [ ]:
# === Visualize Retrieval Results ===
import numpy as np
import matplotlib.pyplot as plt
from src.utils.preprocessing import load_image

def visualize_retrieval(query_idx, labels, paths, descs, top_k=5):
    """Show query image and top-k retrieved images."""
    
    # Compute similarity
    query_desc = descs[query_idx:query_idx+1]  # [1, D]
    sims = (descs @ query_desc.T).squeeze()    # [N]
    sims[query_idx] = -1  # Exclude self
    
    # Get top-k indices
    top_indices = np.argsort(-sims)[:top_k]
    
    # Create figure
    fig, axes = plt.subplots(1, top_k + 1, figsize=(3*(top_k+1), 4))
    
    # Show query
    query_img = load_image(paths[query_idx])
    axes[0].imshow(query_img, cmap='gray')
    axes[0].set_title(f'Query\nWriter: {labels[query_idx]}', fontweight='bold', color='blue')
    axes[0].axis('off')
    
    # Show retrieved images
    query_writer = labels[query_idx]
    for i, idx in enumerate(top_indices):
        img = load_image(paths[idx])
        axes[i+1].imshow(img, cmap='gray')
        
        writer = labels[idx]
        is_correct = (writer == query_writer)
        color = 'green' if is_correct else 'red'
        marker = '✓' if is_correct else '✗'
        
        axes[i+1].set_title(f'Rank {i+1} {marker}\nWriter: {writer}\nSim: {sims[idx]:.3f}', 
                           fontweight='bold', color=color)
        axes[i+1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize a few queries
if 'descs' in dir():
    logger.info('🖼️ Visualizing sample retrievals...')
    for query_idx in [0, 10, 50]:
        if query_idx < len(paths):
            visualize_retrieval(query_idx, labels, paths, descs, top_k=5)
